In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
# Create a Model Class that inherits nn.Module
class Model(nn.Module):
  # Input layer (4 features of the flower) -->
  # Hidden Layer1 (number of neurons) -->
  # H2 (n) -->
  # output (3 classes of iris flowers)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [27]:
#Pick ad manual seed for randomization
torch.manual_seed(41)
#Create an instance of model
model = Model()

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
url ='https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
my_df =pd.read_csv(url)

In [30]:
my_df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [31]:
my_df['variety']= my_df['variety'].replace('Setosa',0.0)
my_df['variety']= my_df['variety'].replace('Versicolor',1.0)
my_df['variety']= my_df['variety'].replace('Virginica',2.0)
my_df



C:\Users\Swapnil Gaware\AppData\Local\Temp\ipykernel_13216\3845813758.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['variety']= my_df['variety'].replace('Virginica',2.0)


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [32]:
#Train test split  Set X,y
X= my_df.drop('variety',axis=1)
y=my_df['variety']

In [33]:
#convert this to numpy arrays
X=X.values
y=y.values

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
#Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=41)

In [36]:
X_train= torch.FloatTensor(X_train)
X_test= torch.FloatTensor(X_test)


In [37]:
y_train= torch.LongTensor(y_train)
y_test= torch.LongTensor(y_test)

In [38]:
#set the criterion of the model to measure the error, how far off the predictions are from the data
criterion =nn.CrossEntropyLoss()
#choode Adam optimizer, learning rate(if error doesn't go down after a bunch of iterations(epochs), lowe our learning rate)
optimizer= torch.optim.Adam(model.parameters(), lr=0.01)
 

In [39]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [40]:
#Train our model
#Epoch? (one run thru all the training data in out network)
epochs =100
losses=[]
for i in range(epochs):
    #Go forward and get a prediction
    y_pred= model.forward(X_train)

    #Measure the loss/error gonna be high at first
    loss= criterion(y_pred,y_train) #predicted values vs the y_train values

    #keep Track of ou losses
    losses.append(loss.detach().numpy)